In [10]:
#!/usr/bin/env python3
"""
Compute Satlab SRS-4 (S-band) link margin vs. elevation for two altitudes and
export both a PNG plot and a CSV table.

Inputs pulled from CSV (Satlab SRS-4 column):
- selected_carrier_frequency [GHz]
- tx_power [W]
- peak_boresight_antenna_gain [dBi]
- interconnect_losses_gains [dB]  (positive = loss)
- atmospheric_attenuation, rain_attenuation, scintillation_loss,
  polarisation_loss, other_propagation_losses [dB]
- g_t [dB/K]
- data_rate_in_db_hz [dB-Hz]
- codmod_inferred_required_eb_n0 [dB]
- other_overall_link_losses, gain_rolloff_loss_at_pointing_error [dB]

Outputs:
- ./Satlab_SRS-4_S-band_link_margin_vs_elevation.csv
- ./Satlab_SRS-4_S-band_link_margin_vs_elevation.png
"""

import os
import math
import argparse
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

EARTH_RADIUS_KM = 6371.0
TARGET_COLUMN = "Satlab SRS-4"  # <-- column in s_down_info.csv

# -------- helpers --------
def to_float(x):
    try:
        return float(str(x).replace("%", "").strip())
    except Exception:
        return None

def resolve_csv_path(cli_csv: str | None):
    # Prefer an explicit path if provided
    if cli_csv and cli_csv.lower().endswith(".csv") and Path(cli_csv).is_file():
        return cli_csv
    # Defaults for S-band
    for p in [
        "./s_down_info.csv",
        os.path.expanduser("~/Downloads/s_down_info.csv"),
        "/mnt/data/s_down_info.csv",
    ]:
        if Path(p).is_file():
            return p
    raise FileNotFoundError(
        "CSV not found. Pass a path explicitly (e.g. --csv C:\\path\\s_down_info.csv) "
        "or place s_down_info.csv in the current folder."
    )

def get_var(df: pd.DataFrame, varname: str, default=None):
    s = df.loc[df["python_variable"] == varname, TARGET_COLUMN]
    return to_float(s.iloc[0]) if not s.empty else default

def slant_range_km(alt_km: float, elev_deg: float) -> float:
    e = math.radians(elev_deg)
    R = EARTH_RADIUS_KM
    H = R + alt_km
    return -R * math.sin(e) + math.sqrt((R * math.sin(e)) ** 2 + H ** 2 - R ** 2)

def fspl_db(range_km: float, freq_ghz: float) -> float:
    # Standard: 92.45 + 20log10(R[km]) + 20log10(f[GHz])
    return 92.45 + 20 * math.log10(range_km) + 20 * math.log10(freq_ghz)

def eirp_dbw(P_w: float, G_tx_dbi: float, L_tx_db: float) -> float:
    return 10 * math.log10(P_w) + (G_tx_dbi or 0.0) - (L_tx_db or 0.0)

def cn0_dbhz(eirp_dbw: float, L_total_db: float, G_T_dbk: float,
             k_val: float = 1.38e-23, extra_losses_db: float = 0.0) -> float:
    k_db = 10 * math.log10(k_val)
    return eirp_dbw - L_total_db + (G_T_dbk or 0.0) - k_db - (extra_losses_db or 0.0)

# -------- main compute --------
def compute_table(csv_path: str,
                  altitudes=(500, 600),
                  elevations=(0, 5, 10, 15, 20)) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # constants from CSV
    f_ghz = get_var(df, "selected_carrier_frequency")
    tx_power_w = get_var(df, "tx_power")
    Gtx_dbi = get_var(df, "peak_boresight_antenna_gain")
    Ltx_db = get_var(df, "interconnect_losses_gains", 0.0)

    atm = get_var(df, "atmospheric_attenuation", 0.0)
    rain = get_var(df, "rain_attenuation", 0.0)
    scint = get_var(df, "scintillation_loss", 0.0)
    pol = get_var(df, "polarisation_loss", 0.0)
    other_prop = get_var(df, "other_propagation_losses", 0.0)

    G_T = get_var(df, "g_t")
    Rb_dBHz = get_var(df, "data_rate_in_db_hz")
    EbN0_req = get_var(df, "codmod_inferred_required_eb_n0", 0.0)
    other_overall = get_var(df, "other_overall_link_losses", 0.0)
    rolloff = get_var(df, "gain_rolloff_loss_at_pointing_error", 0.0)

    rows = []
    for alt in altitudes:
        for el in elevations:
            sr_km = slant_range_km(alt, el)
            Lfs = fspl_db(sr_km, f_ghz)
            Ltotal = Lfs + atm + rain + scint + pol + other_prop
            eirp = eirp_dbw(tx_power_w, Gtx_dbi, Ltx_db)
            cn0 = cn0_dbhz(eirp, Ltotal, G_T, 1.38e-23, other_overall + rolloff)
            ebn0 = cn0 - Rb_dBHz
            margin = ebn0 - EbN0_req

            rows.append({
                "orbit_altitude_km": alt,
                "elevation_deg": el,
                "slant_range_km": sr_km,
                "fspl_db": Lfs,
                "total_prop_loss_db": Ltotal,
                "eirp_dbw": eirp,
                "cn0_dbhz": cn0,
                "ebn0_db": ebn0,
                "link_margin_db": margin,
            })

    return pd.DataFrame(rows)

def make_plot(df_out: pd.DataFrame, png_path: str):
    plt.figure(figsize=(8, 6))
    for alt in sorted(df_out["orbit_altitude_km"].unique()):
        subset = df_out[df_out["orbit_altitude_km"] == alt].sort_values("elevation_deg")
        plt.plot(subset["elevation_deg"], subset["link_margin_db"], marker="o", label=f"{int(alt)} km")
    plt.title("Link Margin vs Elevation — Satlab SRS-4 (S-band)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Link Margin (dB)")
    plt.xticks(sorted(df_out["elevation_deg"].unique()))
    plt.grid(True)
    plt.legend()
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()

def main():
    parser = argparse.ArgumentParser(description="Generate S-band Link Margin vs Elevation plot and CSV for Satlab SRS-4.")
    parser.add_argument("--csv", help="Path to s_down_info.csv")
    parser.add_argument("--out-csv", default="Satlab_SRS-4_S-band_link_margin_vs_elevation.csv")
    parser.add_argument("--out-png", default="Satlab_SRS-4_S-band_link_margin_vs_elevation.png")
    parser.add_argument("--alts", nargs="*", type=float, default=[500, 600],
                        help="Altitudes in km (space-separated)")
    parser.add_argument("--elevs", nargs="*", type=float, default=[0, 5, 10, 15, 20],
                        help="Elevations in degrees (space-separated)")

    args, _ = parser.parse_known_args()
    csv_path = resolve_csv_path(args.csv)

    df_out = compute_table(csv_path, altitudes=args.alts, elevations=args.elevs)
    df_out.to_csv(args.out_csv, index=False)
    make_plot(df_out, args.out_png)

    print(f"Wrote CSV -> {args.out_csv}")
    print(f"Wrote PNG -> {args.out_png}")

if __name__ == "__main__":
    main()


Wrote CSV -> Satlab_SRS-4_S-band_link_margin_vs_elevation.csv
Wrote PNG -> Satlab_SRS-4_S-band_link_margin_vs_elevation.png


In [11]:
#!/usr/bin/env python3
"""
Compute Satlab SRS-4 (S-band) link margin vs elevation for two altitudes and
export both a PNG plot and a CSV table.

Fixes vs earlier:
- EIRP uses interconnect_losses_gains as-is (negative = loss).
- Data rate in dB-Hz is derived from Supported user data rate if the sheet value is missing or implausible.
- Required Eb/N0 can be passed via --ebn0-req when not present in the CSV.

Outputs:
- ./Satlab_SRS-4_S-band_link_margin_vs_elevation.csv
- ./Satlab_SRS-4_S-band_link_margin_vs_elevation.png
"""

import os
import math
import argparse
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

EARTH_RADIUS_KM = 6371.0
TARGET_COLUMN = "Satlab SRS-4"  # s_down_info.csv column

# ---------------- helpers ----------------
def to_float(x):
    try:
        return float(str(x).replace("%", "").strip())
    except Exception:
        return None

def resolve_csv_path(cli_csv: str | None):
    if cli_csv and cli_csv.lower().endswith(".csv") and Path(cli_csv).is_file():
        return cli_csv
    for p in ["./s_down_info.csv",
              os.path.expanduser("~/Downloads/s_down_info.csv"),
              "/mnt/data/s_down_info.csv"]:
        if Path(p).is_file():
            return p
    raise FileNotFoundError("CSV not found. Pass --csv or place s_down_info.csv in the current folder.")

def get_row(df: pd.DataFrame, varname: str):
    s = df[df["python_variable"] == varname]
    return s.iloc[0] if not s.empty else None

def get_val(df: pd.DataFrame, varname: str, default=None):
    row = get_row(df, varname)
    return to_float(row[TARGET_COLUMN]) if row is not None else default

def get_units(df: pd.DataFrame, varname: str):
    row = get_row(df, varname)
    return str(row["Units"]).strip() if row is not None and "Units" in row.index else ""

def freq_to_ghz(val, units: str):
    units = (units or "").lower()
    if val is None:
        return None
    if "ghz" in units:
        return val
    if "mhz" in units:
        return val / 1e3
    if "hz" in units and "ghz" not in units and "mhz" not in units:
        return val / 1e9
    # assume GHz if unspecified
    return val

def slant_range_km(alt_km: float, elev_deg: float) -> float:
    e = math.radians(elev_deg)
    R = EARTH_RADIUS_KM
    H = R + alt_km
    return -R * math.sin(e) + math.sqrt((R * math.sin(e)) ** 2 + H ** 2 - R ** 2)

def fspl_db(range_km: float, freq_ghz: float) -> float:
    return 92.45 + 20 * math.log10(range_km) + 20 * math.log10(freq_ghz)

def eirp_dbw(P_w: float, G_tx_dbi: float, interconn_db: float) -> float:
    # interconnect_db is used as-is: negative=loss, positive=gain
    return 10 * math.log10(P_w) + (G_tx_dbi or 0.0) + (interconn_db or 0.0)

def cn0_dbhz(eirp_dbw_val: float, L_total_db: float, G_T_dbk: float,
             k_val: float = 1.38e-23, extra_losses_db: float = 0.0) -> float:
    k_db = 10 * math.log10(k_val)
    return eirp_dbw_val - L_total_db + (G_T_dbk or 0.0) - k_db - (extra_losses_db or 0.0)

def rb_dbhz_from_supported(df: pd.DataFrame):
    # Use Supported user data rate if present (Mbps/Mb/s)
    r = get_val(df, "supported_user_data_rate")
    units = get_units(df, "supported_user_data_rate").lower()
    if r is None:
        return None
    # interpret units
    if "mb" in units:  # Mbps or Mb/s
        bps = r * 1e6
    elif "kb" in units:
        bps = r * 1e3
    elif "b/s" in units or "bps" in units:
        bps = r
    else:
        # assume Mbps if unspecified
        bps = r * 1e6
    return 10 * math.log10(bps)

# --------------- main compute ---------------
def compute_table(csv_path: str,
                  altitudes=(500, 600),
                  elevations=(0, 5, 10, 15, 20),
                  ebn0_required_cli: float | None = None) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # frequency (unit-robust)
    f_raw = get_val(df, "selected_carrier_frequency")
    f_units = get_units(df, "selected_carrier_frequency")
    f_ghz = freq_to_ghz(f_raw, f_units)

    # TX chain
    tx_power_w = get_val(df, "tx_power")
    Gtx_dbi = get_val(df, "peak_boresight_antenna_gain")
    interconn_db = get_val(df, "interconnect_losses_gains", 0.0)

    # propagation losses
    atm = get_val(df, "atmospheric_attenuation", 0.0) or 0.0
    rain = get_val(df, "rain_attenuation", 0.0) or 0.0
    scint = get_val(df, "scintillation_loss", 0.0) or 0.0
    pol = get_val(df, "polarisation_loss", 0.0) or 0.0
    other_prop = get_val(df, "other_propagation_losses", 0.0) or 0.0

    # receiver + misc
    G_T = get_val(df, "g_t")
    Rb_dBHz_sheet = get_val(df, "data_rate_in_db_hz")
    Rb_dBHz_infer = rb_dbhz_from_supported(df)
    # choose: if sheet value is missing or implausible (< 20 dB-Hz), use inferred
    Rb_dBHz = Rb_dBHz_sheet if (Rb_dBHz_sheet is not None and Rb_dBHz_sheet > 20) else Rb_dBHz_infer

    EbN0_req_sheet = get_val(df, "codmod_inferred_required_eb_n0")
    EbN0_req = ebn0_required_cli if ebn0_required_cli is not None else (EbN0_req_sheet if EbN0_req_sheet is not None else 0.0)

    other_overall = get_val(df, "other_overall_link_losses", 0.0) or 0.0
    rolloff = get_val(df, "gain_rolloff_loss_at_pointing_error", 0.0) or 0.0

    rows = []
    for alt in altitudes:
        for el in elevations:
            sr_km = slant_range_km(alt, el)
            Lfs = fspl_db(sr_km, f_ghz)
            Ltotal = Lfs + atm + rain + scint + pol + other_prop
            eirp = eirp_dbw(tx_power_w, Gtx_dbi, interconn_db)
            cn0 = cn0_dbhz(eirp, Ltotal, G_T, 1.38e-23, other_overall + rolloff)
            ebn0 = cn0 - Rb_dBHz if Rb_dBHz is not None else None
            margin = (ebn0 - EbN0_req) if (ebn0 is not None and EbN0_req is not None) else None

            rows.append({
                "orbit_altitude_km": alt,
                "elevation_deg": el,
                "slant_range_km": sr_km,
                "fspl_db": Lfs,
                "total_prop_loss_db": Ltotal,
                "eirp_dbw": eirp,
                "cn0_dbhz": cn0,
                "rb_dbhz_used": Rb_dBHz,
                "ebn0_db": ebn0,
                "ebn0_required_db": EbN0_req,
                "link_margin_db": margin,
            })

    return pd.DataFrame(rows)

def make_plot(df_out: pd.DataFrame, png_path: str):
    plt.figure(figsize=(8, 6))
    for alt in sorted(df_out["orbit_altitude_km"].unique()):
        subset = df_out[df_out["orbit_altitude_km"] == alt].sort_values("elevation_deg")
        plt.plot(subset["elevation_deg"], subset["link_margin_db"], marker="o", label=f"{int(alt)} km")
    plt.title("Link Margin vs Elevation — Satlab SRS-4 (S-band)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Link Margin (dB)")
    plt.xticks(sorted(df_out["elevation_deg"].unique()))
    plt.grid(True)
    plt.legend()
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()

def main():
    parser = argparse.ArgumentParser(description="Generate S-band Link Margin vs Elevation (Satlab SRS-4).")
    parser.add_argument("--csv", help="Path to s_down_info.csv")
    parser.add_argument("--out-csv", default="Satlab_SRS-4_S-band_link_margin_vs_elevation.csv")
    parser.add_argument("--out-png", default="Satlab_SRS-4_S-band_link_margin_vs_elevation.png")
    parser.add_argument("--alts", nargs="*", type=float, default=[500, 600], help="Altitudes in km")
    parser.add_argument("--elevs", nargs="*", type=float, default=[0, 5, 10, 15, 20], help="Elevations in degrees")
    parser.add_argument("--ebn0-req", type=float, default=None, help="Required Eb/N0 in dB (override if missing in CSV)")
    args, _ = parser.parse_known_args()

    csv_path = resolve_csv_path(args.csv)
    df_out = compute_table(csv_path, altitudes=args.alts, elevations=args.elevs, ebn0_required_cli=args.ebn0_req)
    df_out.to_csv(args.out_csv, index=False)

    # Only plot if we have a margin (i.e., ebn0 and requirement known)
    if df_out["link_margin_db"].notna().any():
        make_plot(df_out, args.out_png)
        print(f"Wrote PNG -> {args.out_png}")
    else:
        print("Note: Required Eb/N0 not provided; plot skipped because margin is undefined. "
              "Pass --ebn0-req to enable plotting.")

    print(f"Wrote CSV -> {args.out_csv}")

if __name__ == "__main__":
    main()


Wrote PNG -> Satlab_SRS-4_S-band_link_margin_vs_elevation.png
Wrote CSV -> Satlab_SRS-4_S-band_link_margin_vs_elevation.csv


In [12]:
#!/usr/bin/env python3
"""
Notebook/CLI friendly pass data budget generator — Satlab SRS-4 (S-band).
Outputs in MB (Megabytes, decimal; 1 MB = 1e6 bytes).

- Auto mode (no args, great for notebooks):
  * looks for Satlab_SRS-4_S-band_link_margin_vs_elevation.csv and s_down_info.csv
    in ./ or /mnt/data
  * finds all altitudes in the link CSV
  * writes pass_data_budget_<alt>km.csv for each altitude

- CLI mode (same flags as before):
  --link-csv <Satlab_SRS-4_S-band_link_margin_vs_elevation.csv>
  --info-csv <s_down_info.csv>
  --alt-km <500> --emin <min_elev_deg> --margin-min <dB> --out-csv <output.csv>
"""

import argparse, math, sys, os
from pathlib import Path
import pandas as pd

MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6371.0        # km
INFO_COLUMN = "Satlab SRS-4"  # column in s_down_info.csv

def find_file(name: str):
    for p in [Path(name), Path("/mnt/data")/name]:
        if p.is_file():
            return str(p)
    return None

def get_supported_rate(info_csv: str):
    df = pd.read_csv(info_csv)
    # Try python_variable first, then Parameter text
    s = df.loc[df["python_variable"].astype(str) == "supported_user_data_rate"]
    if s.empty:
        s = df[df["Parameter"].astype(str).str.contains("Supported user data rate", case=False, na=False)]
    if s.empty:
        raise ValueError("Could not find 'Supported user data rate' in s_down_info.csv")
    val = s.iloc[0][INFO_COLUMN]
    units = s.iloc[0]["Units"] if "Units" in df.columns else ""
    rate_val = float(str(val).strip())
    unit_str = (units or "").lower()
    if "mb" in unit_str:      bps, pretty = rate_val * 1e6, "Mb/s"
    elif "kb" in unit_str:    bps, pretty = rate_val * 1e3, "kb/s"
    elif "b/s" in unit_str or "bps" in unit_str:
                              bps, pretty = rate_val, "b/s"
    else:                     bps, pretty = rate_val * 1e6, "Mb/s (assumed)"
    return bps, pretty

def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

def build_budget(link_csv: str, info_csv: str, alt_km: float,
                 emin: float | None, margin_min: float, out_csv: str) -> pd.DataFrame:
    df = pd.read_csv(link_csv)
    need = {"orbit_altitude_km","elevation_deg","link_margin_db","slant_range_km"}
    if not need.issubset(df.columns):
        raise ValueError(f"link CSV missing columns: {need - set(df.columns)}")

    dfa = df[df["orbit_altitude_km"].astype(float) == float(alt_km)].copy()
    if dfa.empty:
        raise ValueError(f"No rows for orbit_altitude_km={alt_km}")
    dfa.sort_values("elevation_deg", inplace=True)

    bps, unit_str = get_supported_rate(info_csv)
    if emin is None:
        emin = float(dfa["elevation_deg"].min())

    e = dfa["elevation_deg"].to_list()
    m = dfa["link_margin_db"].to_list()
    r = dfa["slant_range_km"].to_list()
    theta = [central_angle_from_slant(x, alt_km) for x in r]
    n = mean_motion_rad_s(alt_km)

    rows = []
    for i in range(len(e)-1):
        e0, e1 = e[i], e[i+1]
        m0, m1 = m[i], m[i+1]
        th0, th1 = theta[i], theta[i+1]

        if max(e0, e1) < emin:
            continue
        e_start, e_end = max(min(e0, e1), emin), max(e0, e1)
        if e_end <= e_start:
            continue

        def lerp(x0, y0, x1, y1, x):
            t = (x - x0) / (x1 - x0)
            return y0 + t * (y1 - y0)

        # ensure ascending for interpolation
        if e1 < e0:
            e0, e1 = e1, e0
            m0, m1 = m1, m0
            th0, th1 = th1, th0

        th_start = lerp(e0, th0, e1, th1, e_start)
        th_end   = lerp(e0, th0, e1, th1, e_end)
        m_start  = lerp(e0, m0,  e1, m1,  e_start)
        m_end    = lerp(e0, m0,  e1, m1,  e_end)

        # time across BOTH sides of pass (rise + set)
        dtheta = max(0.0, th_start - th_end)
        bin_time_s = 2.0 * dtheta / n

        # fraction of this bin that meets the margin threshold
        usable_fraction = 1.0
        if (m_start < margin_min) and (m_end < margin_min):
            usable_fraction = 0.0
        elif (m_start < margin_min) or (m_end < margin_min):
            if m_end != m_start:
                e_cross = e_start + (margin_min - m_start) * (e_end - e_start) / (m_end - m_start)
                usable_fraction = (e_end - e_cross) / (e_end - e_start) if e_start <= e_cross <= e_end else (1.0 if m_end >= margin_min else 0.0)
            else:
                usable_fraction = 0.0 if m_start < margin_min else 1.0

        data_bits = bps * bin_time_s * usable_fraction
        data_MB   = data_bits / (8 * 1e6)   # MB (decimal)
        # keep MiB as optional reference if you like:
        data_MiB  = data_bits / (8 * 1024**2)

        rows.append({
            "alt_km": float(alt_km),
            "elev_start_deg": float(e_start),
            "elev_end_deg": float(e_end),
            "theta_start_deg": math.degrees(th_start),
            "theta_end_deg": math.degrees(th_end),
            "bin_time_s": bin_time_s,
            "margin_start_db": m_start,
            "margin_end_db": m_end,
            "usable_fraction": usable_fraction,
            "data_bits": data_bits,
            "data_MB": data_MB,     # <-- main output unit
            "data_MiB": data_MiB,   # (optional, binary)
            "rate_bps": bps,
            "rate_unit": unit_str,
        })

    out = pd.DataFrame(rows)
    out.to_csv(out_csv, index=False)
    print(f"[{int(alt_km)} km] time≥thr: {out['bin_time_s'].sum():.1f}s  data: {out['data_MB'].sum():.1f} MB")
    return out

def main():
    parser = argparse.ArgumentParser(description="Create S-band pass data budget for Satlab SRS-4 (MB output).")
    parser.add_argument("--link-csv", required=False, help="Path to Satlab_SRS-4_S-band_link_margin_vs_elevation.csv")
    parser.add_argument("--info-csv", required=False, help="Path to s_down_info.csv")
    parser.add_argument("--alt-km", type=float, required=False, help="Orbit altitude to use")
    parser.add_argument("--emin", type=float, default=None, help="Minimum elevation (deg)")
    parser.add_argument("--margin-min", type=float, default=0.0, help="Minimum link margin counted (dB)")
    parser.add_argument("--out-csv", required=False, help="Output CSV path")
    args, _ = parser.parse_known_args()

    # Auto mode (nice for notebooks)
    if len(sys.argv) == 1 or (args.link_csv is None and args.info_csv is None):
        link_csv = find_file("Satlab_SRS-4_S-band_link_margin_vs_elevation.csv")
        info_csv = find_file("s_down_info.csv")
        if not link_csv or not info_csv:
            raise SystemExit("Auto-mode couldn't find the CSVs. Put them next to the script or pass --link-csv/--info-csv.")
        df = pd.read_csv(link_csv)
        alts = sorted(set(df["orbit_altitude_km"]))
        for alt in alts:
            out_csv = f"pass_data_budget_{int(alt)}km.csv"
            build_budget(link_csv, info_csv, float(alt), args.emin, args.margin_min, out_csv)
        return

    # CLI mode
    if not all([args.link_csv, args.info_csv, args.alt_km, args.out_csv]):
        parser.error("the following arguments are required: --link-csv, --info-csv, --alt-km, --out-csv")

    build_budget(args.link_csv, args.info_csv, args.alt_km, args.emin, args.margin_min, args.out_csv)

if __name__ == "__main__":
    main()


[500 km] time≥thr: 396.9s  data: 312.6 MB
[600 km] time≥thr: 422.5s  data: 332.7 MB


In [13]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# --- Config (tweak if you want) ---
OUTDIR = Path(".")
FILES = None   # e.g. ["pass_data_budget_500km.csv", "pass_data_budget_600km.csv"]; None => auto-find
# ----------------------------------

def find_pass_files():
    pats = [Path(".").glob("pass_data_budget_*km.csv"),
            Path("/mnt/data").glob("pass_data_budget_*km.csv")]
    files = []
    for g in pats:
        for p in g:
            if p.is_file() and p.suffix.lower() == ".csv" and "pass_data_budget_" in p.name:
                files.append(p.resolve())
    # de-dup while preserving order
    seen, uniq = set(), []
    for p in files:
        if p not in seen:
            uniq.append(p); seen.add(p)
    return uniq

def pick_mb_series(df: pd.DataFrame) -> pd.Series:
    """Return a Series in MB (decimal, 1 MB = 1e6 bytes) from best available column."""
    cols = set(df.columns)
    if "data_MB" in cols:
        return df["data_MB"]
    if "data_Mb" in cols:          # megabits -> MB
        return df["data_Mb"] / 8.0
    if "data_MiB" in cols:         # MiB -> MB
        return df["data_MiB"] * (1024**2) / 1e6
    if "data_bits" in cols:        # bits -> MB
        return df["data_bits"] / (8 * 1e6)
    raise ValueError("No data_MB/data_Mb/data_MiB/data_bits column found.")

def load_one_pass(path: Path):
    df = pd.read_csv(path)
    required = {"bin_time_s", "alt_km"}
    missing = required - set(df.columns)
    if missing:
        print(f"⚠️  Skipping {path.name}: missing {missing}")
        return None, None

    # Use the file as-is (passive, no scaling)
    df = df.copy()
    df["cum_time_s"] = df["bin_time_s"].cumsum()
    mb = pick_mb_series(df)
    df["cum_MB"] = mb.cumsum()

    alt = int(round(df["alt_km"].iloc[0]))
    summary = {
        "file": path.name,
        "alt_km": alt,
        "total_time_s": float(df["bin_time_s"].sum()),
        "total_MB": float(mb.sum()),
        "bins": len(df),
    }
    return df, summary

# Discover files
paths = [Path(p) for p in FILES] if FILES else find_pass_files()
if not paths:
    raise SystemExit("No pass_data_budget_*km.csv files found. Generate the S-band budgets first.")

OUTDIR.mkdir(parents=True, exist_ok=True)

curves, summaries = [], []
for p in paths:
    d, s = load_one_pass(p)
    if d is None:
        continue
    curves.append(d)
    summaries.append(s)
    print(f"[{s['alt_km']} km] duration: {s['total_time_s']:.1f}s | data: {s['total_MB']:.1f} MB | bins: {s['bins']}")

# Combined cumulative MB vs time (passive, no scaling)
if curves:
    plt.figure(figsize=(8,6))
    for d in curves:
        alt = int(round(d["alt_km"].iloc[0]))
        plt.plot(d["cum_time_s"], d["cum_MB"], marker="o", label=f"{alt} km")
    plt.title("Cumulative Data vs Time (Passive S-band)")
    plt.xlabel("Time (s)")
    plt.ylabel("Cumulative Data (MB)")
    plt.grid(True)
    plt.legend()
    png_path = OUTDIR / "pass_data_budget_cumulative.png"
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()
    print(f"Wrote cumulative plot → {png_path}")

# Optional: show a compact summary table right in the notebook
pd.DataFrame(summaries)


[500 km] duration: 396.9s | data: 312.6 MB | bins: 4
[600 km] duration: 422.5s | data: 332.7 MB | bins: 4
Wrote cumulative plot → pass_data_budget_cumulative.png


,file,alt_km,total_time_s,total_MB,bins
0,pass_data_budget_500km.csv,500,396.905248,312.562883,4
1,pass_data_budget_600km.csv,600,422.499347,332.718236,4


In [14]:
# Notebook-friendly: S-band "passes per day" planner (Satlab SRS-4)
# - Scenarios CSV: sums "Payload ..." rows per scenario (MB/day)
# - Per-pass capacity (MB) per altitude:
#     * Prefer precomputed pass_data_budget_*km.csv (uses data_MB)
#     * Else compute from Satlab_SRS-4_S-band_link_margin_vs_elevation.csv + s_down_info.csv
# - Output: passes_per_day_plan.csv + passes_per_day_plan.png (grouped bar chart)

from pathlib import Path
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------- Config you can tweak ----------------
SCENARIOS_CSV = None  # e.g. "data_budget_scenarios.csv"; None => auto-find in . or /mnt/data
LINK_CSV      = None  # e.g. "Satlab_SRS-4_S-band_link_margin_vs_elevation.csv"; None => auto-find
INFO_CSV      = None  # e.g. "s_down_info.csv"; None => auto-find
INCLUDE_PLATFORM = False  # if True, include "Platform" rows in scenario totals
MARGIN_MIN_DB = 0.0       # usability threshold for link margin (0 dB default)
OUTDIR = Path(".")        # where to write outputs
# ------------------------------------------------------

INFO_COLUMN = "Satlab SRS-4"  # column used in s_down_info.csv

def find_default(name: str):
    """Find a file named 'name' in CWD or /mnt/data."""
    p1 = Path(name)
    p2 = Path("/mnt/data") / name
    if p1.is_file():
        return str(p1)
    if p2.is_file():
        return str(p2)
    return None

def find_pass_budget_files():
    """Find pass_data_budget_*km.csv files in CWD or /mnt/data (de-duplicated)."""
    cands = list(Path(".").glob("pass_data_budget_*km.csv")) + list(Path("/mnt/data").glob("pass_data_budget_*km.csv"))
    seen, out = set(), []
    for p in cands:
        rp = p.resolve()
        if rp not in seen:
            out.append(rp)
            seen.add(rp)
    return out

def pick_mb_series(df: pd.DataFrame) -> pd.Series:
    """Return MB (decimal) for a pass_data_budget file from best available column."""
    cols = set(df.columns)
    if "data_MB" in cols:  # Megabytes (decimal)
        return df["data_MB"]
    if "data_Mb" in cols:  # megabits -> MB
        return df["data_Mb"] / 8.0
    if "data_MiB" in cols: # MiB -> MB (binary to decimal)
        return df["data_MiB"] * (1024**2) / 1e6
    if "data_bits" in cols:
        return df["data_bits"] / (8 * 1e6)
    raise ValueError("No data_MB/data_Mb/data_MiB/data_bits column found.")

def per_pass_mb_from_budget(path: Path) -> tuple[int, float]:
    """Return (alt_km, per_pass_MB) from a pass_data_budget_*km.csv file."""
    df = pd.read_csv(path)
    if "alt_km" not in df.columns:
        raise ValueError(f"{path.name} missing 'alt_km'")
    alt = int(round(df["alt_km"].iloc[0]))
    total_MB = float(pick_mb_series(df).sum())
    return alt, total_MB

# ---------- Geometry helpers for fallback compute ----------
MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6371.0

def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

def _to_float(x):
    try:
        return float(str(x).replace("%","").strip())
    except Exception:
        return None

def supported_rate_bps(info_csv: str) -> float:
    """Read Supported user data rate from s_down_info.csv (Satlab SRS-4 column)."""
    df = pd.read_csv(info_csv)
    s = df.loc[df["python_variable"].astype(str)=="supported_user_data_rate"]
    if s.empty:
        s = df[df["Parameter"].astype(str).str.contains("Supported user data rate", case=False, na=False)]
    if s.empty:
        raise ValueError("Supported user data rate not found in s_down_info.csv")
    val = _to_float(s.iloc[0][INFO_COLUMN])
    units = str(s.iloc[0]["Units"]).lower() if "Units" in s.columns else ""
    if "mb" in units:  return val * 1e6
    if "kb" in units:  return val * 1e3
    if "b/s" in units or "bps" in units: return val
    return val * 1e6  # assume Mbps

def per_pass_mb_from_link(link_csv: str, info_csv: str, margin_min_db: float=0.0) -> dict[int, float]:
    """Compute per-pass MB per altitude directly from link curve + supported rate."""
    df = pd.read_csv(link_csv)
    bps = supported_rate_bps(info_csv)
    alts = sorted(set(df["orbit_altitude_km"]))
    out = {}
    for alt in alts:
        dfa = df[df["orbit_altitude_km"]==alt].sort_values("elevation_deg").reset_index(drop=True)
        e = dfa["elevation_deg"].tolist()
        m = dfa["link_margin_db"].tolist()
        r = dfa["slant_range_km"].tolist()
        theta = [central_angle_from_slant(x, alt) for x in r]
        n = mean_motion_rad_s(alt)
        total_bits = 0.0
        for i in range(len(e)-1):
            e0,e1 = e[i], e[i+1]
            m0,m1 = m[i], m[i+1]
            th0,th1 = theta[i], theta[i+1]
            if e1 < e0:
                e0,e1 = e1,e0; m0,m1 = m1,m0; th0,th1 = th1,th0
            dtheta = max(0.0, th0 - th1)
            bin_time_s = 2.0 * dtheta / n  # rise + set
            # usable fraction by margin threshold
            if (m0 < margin_min_db) and (m1 < margin_min_db):
                frac = 0.0
            elif (m0 >= margin_min_db) and (m1 >= margin_min_db):
                frac = 1.0
            else:
                if m1 != m0:
                    e_cross = e0 + (margin_min_db - m0) * (e1 - e0) / (m1 - m0)
                    frac = (e1 - e_cross) / (e1 - e0) if e0 <= e_cross <= e1 else (1.0 if m1 >= margin_min_db else 0.0)
                else:
                    frac = 1.0 if m0 >= margin_min_db else 0.0
            total_bits += bps * bin_time_s * frac
        out[int(alt)] = total_bits / (8 * 1e6)  # MB (decimal)
    return out

# ---------- Scenarios handling ----------
def load_scenarios(scen_csv: str, include_platform=False):
    df = pd.read_csv(scen_csv)
    if "Item" not in df.columns:
        raise ValueError("Scenarios CSV must have an 'Item' column plus one or more scenario columns.")
    scen_cols = [c for c in df.columns if c.lower() != "item"]
    if not scen_cols:
        raise ValueError("No scenario columns found in scenarios CSV.")
    # Choose rows
    if include_platform:
        mask = df["Item"].astype(str).str.len() > 0
    else:
        mask = df["Item"].astype(str).str.contains("^payload", case=False, na=False)
    sub = df.loc[mask, ["Item"] + scen_cols].copy()
    # Coerce numbers
    for c in scen_cols:
        sub[c] = pd.to_numeric(sub[c], errors="coerce")
    totals = {c: float(sub[c].sum(skipna=True)) for c in scen_cols}  # MB/day
    return scen_cols, totals, sub

# ------------------- Main (notebook) -------------------
# Resolve inputs
scen_csv = SCENARIOS_CSV or find_default("data_budget_scenarios.csv")
if not scen_csv:
    raise SystemExit("Couldn't find data_budget_scenarios.csv. Set SCENARIOS_CSV.")

# Per-pass capacity (MB) per altitude
per_pass_by_alt = {}
budget_files = find_pass_budget_files()
if budget_files:
    for p in budget_files:
        alt, mb = per_pass_mb_from_budget(p)
        per_pass_by_alt[alt] = mb

if not per_pass_by_alt:
    link_csv = LINK_CSV or find_default("Satlab_SRS-4_S-band_link_margin_vs_elevation.csv")
    info_csv = INFO_CSV or find_default("s_down_info.csv")
    if not link_csv or not info_csv:
        raise SystemExit("Need pass_data_budget_*km.csv files OR (Satlab_SRS-4_S-band_link_margin_vs_elevation.csv + s_down_info.csv).")
    per_pass_by_alt = per_pass_mb_from_link(link_csv, info_csv, MARGIN_MIN_DB)

scen_cols, totals_MB, scen_table = load_scenarios(scen_csv, include_platform=INCLUDE_PLATFORM)

# Build summary table: passes per day = ceil(total_MB_per_day / per_pass_MB)
rows = []
for scen in scen_cols:
    total_MB = totals_MB[scen]
    for alt, cap_MB in sorted(per_pass_by_alt.items()):
        passes = math.ceil(total_MB / cap_MB) if cap_MB > 0 else float('inf')
        rows.append({
            "scenario": scen,
            "total_MB_per_day": total_MB,
            "alt_km": int(alt),
            "per_pass_MB": cap_MB,
            "passes_per_day": int(passes),
        })

summary_df = pd.DataFrame(rows).sort_values(["scenario","alt_km"]).reset_index(drop=True)

# --- Save CSV ---
OUTDIR.mkdir(parents=True, exist_ok=True)
out_csv = OUTDIR / "passes_per_day_plan.csv"
summary_df.to_csv(out_csv, index=False)
print(f"Wrote {out_csv}")

# --- Grouped BAR CHART ---
pivot = summary_df.pivot(index="scenario", columns="alt_km", values="passes_per_day").sort_index()

fig, ax = plt.subplots(figsize=(9,6))
alts = sorted(pivot.columns)
x = np.arange(len(pivot.index))

group_width = 0.8
bar_w = group_width / max(1, len(alts))

for i, alt in enumerate(alts):
    offset = (i - (len(alts)-1)/2) * bar_w
    bars = ax.bar(x + offset, pivot[alt].values, width=bar_w, label=f"{alt} km")
    # label bars with integers
    try:
        ax.bar_label(bars, padding=2, fmt="%.0f")
    except Exception:
        pass

ax.set_title("S-band Passes Needed per Day (Satlab SRS-4)")
ax.set_xlabel("Scenario")
ax.set_ylabel("Passes per day (ceil)")
ax.set_xticks(x)
ax.set_xticklabels(pivot.index, rotation=45, ha="right")
ax.grid(True, axis="y")
ax.legend()
fig.tight_layout()

out_png = OUTDIR / "passes_per_day_plan.png"
fig.savefig(out_png, bbox_inches="tight")
plt.close(fig)
print(f"Wrote {out_png}")

# Show the summary table in the notebook
summary_df


Wrote passes_per_day_plan.csv


Wrote passes_per_day_plan.png


,scenario,total_MB_per_day,alt_km,per_pass_MB,passes_per_day
0,High data,1860.0,500,312.562883,6
1,High data,1860.0,600,332.718236,6
2,Low data,1600.0,500,312.562883,6
3,Low data,1600.0,600,332.718236,5
4,Nominal,1100.0,500,312.562883,4
5,Nominal,1100.0,600,332.718236,4
